<a href="https://colab.research.google.com/github/hotyya/kaggle-Journey-to-Springfield/blob/main/kaggle_simsons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Базовая нейронка со свертками без классификатора**

In [ ]:
# ВНИМАНИЕ; версия Pillow установвленна по умолчанию - '7.0.0'
# Эта версия ломает torchvision
import PIL
PIL.__version__

'9.4.0'

In [ ]:
!pip uninstall -y Pillow
!pip install -Iv Pillow==5.3.0

Found existing installation: Pillow 9.4.0
Uninstalling Pillow-9.4.0:
  Successfully uninstalled Pillow-9.4.0
Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 12.5 MB/s eta 0:00:00
  Running command python setup.py egg_info
  /usr/local/lib/python3.10/dist-packages/setuptools/config/setupcfg.py:293: _DeprecatedConfig: Deprecated config in `setup.cfg`
  !!

          ********************************************************************************
          The license_file parameter is deprecated, use license_files instead.

          This deprecation is overdue, please update your project and remove deprecated
          calls to avoid build errors in the future.

          See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
          ********************************************************************************

  !!
    parsed = self.parsers.get(option_name, l

In [ ]:
!pip install -U torch torchvision

In [ ]:
# ИСПРАВЛЕНИЕ:
# Вручную ищем и устанавливаем подходящуюю версию
# https://download.pytorch.org/whl/torch_stable.html
!pip install -q 'https://download.pytorch.org/whl/cu101/torch-1.5.0%2Bcu101-cp36-cp36m-linux_x86_64.whl' torchvision
import torch

In [ ]:
# we will verify that GPU is enabled for this notebook
# following should print: CUDA is available!  Training on GPU ...
#
# if it prints otherwise, then you need to enable GPU:
# from Menu > Runtime > Change Runtime Type > Hardware Accelerator > GPU

import torch
import numpy as np

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


In [ ]:
!nvidia-smi
import torch
torch.cuda.is_available()

Mon Feb 19 02:42:04 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

True

In [ ]:
import pickle
import numpy as np
from skimage import io

from tqdm import tqdm, tqdm_notebook
from PIL import Image
from pathlib import Path

from torchvision import transforms
from multiprocessing.pool import ThreadPool
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

from matplotlib import colors, pyplot as plt
%matplotlib inline

# в sklearn не все гладко, чтобы в colab удобно выводить картинки
# мы будем игнорировать warnings
import warnings
warnings.filterwarnings(action='ignore', category=DeprecationWarning)


/usr/local/lib/python3.10/dist-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 8.1.2. Several security issues (CVE-2021-27921, CVE-2021-25290, CVE-2021-25291, CVE-2021-25293, and more) have been fixed in pillow 8.1.2 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper
/usr/local/lib/python3.10/dist-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [ ]:
# разные режимы датасета
DATA_MODES = ['train', 'val', 'test']
# все изображения будут масштабированы к размеру 224x224 px
RESCALE_SIZE = 224
# работаем на видеокарте
DEVICE = torch.device("cuda")

In [ ]:
class SimpsonsDataset(Dataset):
    """
    Датасет с картинками, который паралельно подгружает их из папок
    производит скалирование и превращение в торчевые тензоры
    """
    def __init__(self, files, mode):
        super().__init__()
        # список файлов для загрузки
        self.files = sorted(files)
        # режим работы
        self.mode = mode

        if self.mode not in DATA_MODES:
            print(f"{self.mode} is not correct; correct modes: {DATA_MODES}")
            raise NameError

        self.len_ = len(self.files)

        self.label_encoder = LabelEncoder()

        if self.mode != 'test':
            self.labels = [path.parent.name for path in self.files]
            self.label_encoder.fit(self.labels)

            with open('label_encoder.pkl', 'wb') as le_dump_file:
                  pickle.dump(self.label_encoder, le_dump_file)

    def __len__(self):
        return self.len_

    def load_sample(self, file):
        image = Image.open(file)
        image.load()
        return image

    def __getitem__(self, index):
        # для преобразования изображений в тензоры PyTorch и нормализации входа
        transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
        ])
        x = self.load_sample(self.files[index])
        x = self._prepare_sample(x)
        x = np.array(x / 255, dtype='float32')
        x = transform(x)
        if self.mode == 'test':
            return x
        else:
            label = self.labels[index]
            label_id = self.label_encoder.transform([label])
            y = label_id.item()
            return x, y

    def _prepare_sample(self, image):
        image = image.resize((RESCALE_SIZE, RESCALE_SIZE))
        return np.array(image)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
!ls /content/gdrive/

MyDrive


In [ ]:
!unzip -q /content/gdrive/MyDrive/journey-springfield.zip

In [ ]:
from pathlib import Path
TRAIN_DIR = Path('train/simpsons_dataset')
TEST_DIR = Path('testset/testset')

train_val_files = sorted(list(TRAIN_DIR.rglob('*.jpg')))
test_files = sorted(list(TEST_DIR.rglob('*.jpg')))
print(len(train_val_files)) #=> 20933
print(len(test_files)) #=> 991

20933
991


In [ ]:
def imshow(inp, title=None, plt_ax=plt, default=False):
    """Imshow для тензоров"""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt_ax.imshow(inp)
    if title is not None:
        plt_ax.set_title(title)
    plt_ax.grid(False)

In [ ]:
from sklearn.model_selection import train_test_split

train_val_labels = [path.parent.name for path in train_val_files]
train_files, val_files = train_test_split(train_val_files, test_size=0.2, \
                                          stratify=train_val_labels)

In [ ]:
val_dataset = SimpsonsDataset(val_files, mode = 'val')
train_dataset = SimpsonsDataset(train_files, mode = 'train')

**Выведение рандомной картинки на экран**

In [ ]:
fig, ax = plt.subplots(nrows=3, ncols=3,figsize=(8, 8), \
                        sharey=True, sharex=True)
for fig_x in ax.flatten():
    random_characters = int(np.random.uniform(0,1000))
    im_val, label = val_dataset[random_characters]
    img_label = " ".join(map(lambda x: x.capitalize(),\
                val_dataset.label_encoder.inverse_transform([label])[0].split('_')))
    imshow(im_val.data.cpu(), \
          title=img_label,plt_ax=fig_x)

**AlexNet**

In [ ]:
class VGGNet_(nn.Module):
  def __init__(self, n_classes):
    super(VGGNet_, self).__init__()
    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 3, padding = 1)
    self.conv2 = nn.Conv2d(in_channels = 64, out_channels = 64, kernel_size = 3, padding = 1)
    self.pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1)
    self.conv4 = nn.Conv2d(in_channels = 128, out_channels = 128, kernel_size = 3, padding = 1)
    self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.conv5 = nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1)
    self.conv6 = nn.Conv2d(in_channels = 256, out_channels = 256, kernel_size = 3, padding = 1)
    self.pool3 = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.conv7 = nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3, padding = 1)
    self.conv8 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 1)
    self.conv9 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 1)
    self.pool4 = nn.MaxPool2d(kernel_size = 2, stride = 2)
    self.conv10 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 1)
    self.conv11 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 1)
    self.conv12 = nn.Conv2d(in_channels = 512, out_channels = 512, kernel_size = 3, padding = 1)
    self.pool5 = nn.MaxPool2d(kernel_size = 2, stride = 2)

    self.Linear1 = nn.Linear(802816, 4096)
    self.drop1 = nn.Dropout(p = 0.5)
    self.Linear2 = nn.Linear(4096, 1024)
    self.drop2 = nn.Dropout(p = 0.5)
    self.Linear3 = nn.Linear(1024, n_classes)

  def forward(self, x):
    x = self.pool1(nn.functional.relu(self.conv2(nn.functional.relu(self.conv1(x)))))
    x = self.pool2(nn.functional.relu(self.conv4(nn.functional.relu(self.conv3(x)))))
    x = self.pool3(nn.functional.relu(self.conv6(nn.functional.relu(self.conv5(x)))))
    x = self.pool4(nn.functional.relu(self.conv8(nn.functional.relu(self.conv7(x)))))
    x = self.pool5(nn.functional.relu(self.conv11(nn.functional.relu(self.conv10(x)))))

    print(x.shape)

    x = x.view(-1, 802816)

    x = self.drop1(nn.functional.relu(self.Linear1(x)))
    x = self.drop2(nn.functional.relu(self.Linear2(x)))
    logits = self.Linear3(x)

    return logits

In [ ]:
class googlenet_(nn.Module):
  def __init__(self, n_classes):
    super(googlenet_, self).__init__()
    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 64, kernel_size = 7, stride = 2)
    self.pool1 = nn.MaxPool2d(kernel_size = 3, stride = 2)
    self.conv2 = nn.Conv2d(in_channels = 64, out_channels = 192, kernel_size = 3, stride = 1)
    self.pool2 = nn.MaxPool2d(kernel_size = 3, stride = 2)

    self.pool3 = nn.MaxPool2d(kernel_size = 3, stride = 2)

    self.pool4 = nn.MaxPool2d(kernel_size = 3, stride = 2)
    self.pool5 = nn.AvgPool2d(kernel_size = 7, stride = 1)

    self.drop1 = nn.Dropout(p = 0.4)
    self.Linear1 = nn.Linear(1024, n_classes)

  def incept(self, x, input_, o_1, o_2, o_3, o_4):
    self.c1 = nn.Conv2d(in_channels = input_, out_channels = o_1, kernel_size = 1)
    self.c2 = nn.Conv2d(in_channels = input_, out_channels = o_2, kernel_size = 3)
    self.c3 = nn.Conv2d(in_channels = input_, out_channels = o_3, kernel_size = 5)
    self.p1 = nn.MaxPool2d(kernel_size = 3)

    x = x.to(DEVICE)

    x1 = nn.functional.relu(self.c1(x)).to(DEVICE)
    x2 = nn.functional.relu(self.c2(x)).to(DEVICE)
    x3 = nn.functional.relu(self.c3(x)).to(DEVICE)
    x4 = self.p1(x).to(DEVICE)

    x = nn.functional.relu(torch.cat([x1, x2, x3, x4], 1)).to(DEVICE)

    return x

  def forward(self, x):
    x = self.pool1(nn.functional.relu(self.conv1(x)))
    x = self.pool2(nn.functional.relu(self.conv2(x)))

    x = nn.functional.relu(self.incept(x, 192, 64, 128, 32, 32))
    x = nn.functional.relu(self.incept(x, 256, 128, 192, 96, 64))
    print(zalupa)
    x = self.pool3(x)

    x = nn.functional.relu(self.incept(x, 480, 192, 224, 48, 64))
    x = nn.functional.relu(self.incept(x, 528, 256, 320, 128, 128))

    x = self.pool4(x)

    x = nn.functional.relu(self.incept(x, 832, 256, 320, 128, 128))
    x = nn.functional.relu(self.incept(x, 832, 384, 384, 128, 128))

    x = self.pool5(x)
    x = x.view(-1, 1024)
    x = self.drop1(x)
    logits = self.Linear1(x)

    return logits

In [ ]:
class AlexNet_(nn.Module):
  def __init__(self, n_classes):
    super(AlexNet_, self).__init__()
    self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 96, kernel_size = 11, stride = 4)
    self.pool1 = nn.MaxPool2d(kernel_size = 3, stride = 2)
    self.conv2 = nn.Conv2d(in_channels = 96, out_channels = 256, kernel_size = 5, padding = 2)
    self.pool2 = nn.MaxPool2d(kernel_size = 3, stride = 2)
    self.conv3 = nn.Conv2d(in_channels = 256, out_channels = 384, kernel_size = 3, padding = 1)
    self.conv4 = nn.Conv2d(in_channels = 384, out_channels = 384, kernel_size = 3, padding = 1)
    self.conv5 = nn.Conv2d(in_channels = 384, out_channels = 256, kernel_size = 3, padding = 1)
    self.pool3 = nn.MaxPool2d(kernel_size = 3, stride = 2)

    self.Linear1 = nn.Linear(6400, 4096)
    self.drop1 = nn.Dropout(p = 0.3)
    self.Linear2 = nn.Linear(4096, 1000)
    self.drop2 = nn.Dropout(p = 0.3)
    self.Linear3 = nn.Linear(1000, n_classes)

  def forward(self, x):
    x = self.pool1(nn.functional.relu(self.conv1(x)))
    x = self.pool2(nn.functional.relu(self.conv2(x)))
    x = nn.functional.relu(self.conv4(nn.functional.relu(self.conv3(x))))
    x = self.pool3(nn.functional.relu(self.conv5(x)))

    x = x.view(-1, 5*5*256)

    x = self.drop1(nn.functional.relu(self.Linear1(x)))
    x = self.drop2(nn.functional.relu(self.Linear2(x)))
    logits = self.Linear3(x)

    return logits

In [ ]:
def fit_epoch(model, train_loader, criterion, optim):
  model.train()
  running_loss = 0.0
  running_corrects = 0
  processed_data = 0

  for inputs, labels in train_loader:
    inputs = inputs.to(DEVICE)
    labels = labels.to(DEVICE)
    optim.zero_grad()

    outp = model(inputs)
    loss = criterion(outp, labels)
    loss.backward()
    optim.step()

    preds = torch.argmax(outp, 1)
    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)
    processed_data += inputs.size(0)

  train_loss = running_loss / processed_data
  train_acc = running_corrects.cpu().numpy() / processed_data

  return train_loss, train_acc

In [ ]:
def eval_epoch(model, val_loader, criterion):
  model.eval()
  running_loss = 0.0
  running_corrects = 0
  processed_data = 0

  for inputs, labels in val_loader:
    inputs = inputs.to(DEVICE)
    labels = labels.to(DEVICE)

    with torch.set_grad_enabled(False):
      outp = model(inputs)
      loss = criterion(outp, labels)
      preds = torch.argmax(outp, 1)

    running_loss += loss.item() * inputs.size(0)
    running_corrects += torch.sum(preds == labels.data)
    processed_data += inputs.size(0)

  val_loss = running_loss / processed_data
  val_acc = running_corrects.double() / processed_data

  return val_loss, val_acc

In [ ]:
def train(model, train_files, val_files, epochs, batch_size, lr):
  train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle = True)
  val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle = False)

  history = []
  log_template = "\nEpoch {ep:03d} train_loss: {t_loss:0.4f} \
  val_loss {v_loss:0.4f} train_acc {t_acc:0.4f} val_acc {v_acc:0.4f}"

  with tqdm(desc = 'epoch', total = epochs) as pbar_outer:
    optim = torch.optim.Adam(model.parameters(), lr)
    criterion = nn.CrossEntropyLoss()

    for epoch in range(epochs):
      train_loss, train_acc = fit_epoch(model, train_loader, criterion, optim)
      print('loss: ', train_loss)

      val_loss, val_acc = eval_epoch(model, val_loader, criterion)
      history.append((train_loss, train_acc, val_loss, val_acc))

      pbar_outer.update(1)
      tqdm.write(log_template.format(ep = epoch + 1, t_loss = train_loss, v_loss = val_loss, t_acc = train_acc, v_acc = val_acc))

  return history

In [ ]:
def predict(model, test_loader):
  with torch.no_grad():
    logits = []

    for inputs in test_loader:
      inputs = inputs.to(DEVICE)
      model.eval()
      outp = model(inputs).cpu()
      logits.append(outp)

  preds = nn.functional.softmax(torch.cat(logits), dim = -1).numpy()

  return preds

**Обучение AlexNet**

In [ ]:
n_classes = len(np.unique(train_val_labels))
alexnet_ = AlexNet_(n_classes).to(DEVICE)

history = train(alexnet_, train_dataset, val_dataset, epochs = 15, batch_size = 128, lr = 1e-4)

epoch:   0%|          | 0/15 [00:00<?, ?it/s]

loss:  2.978886388755679


epoch:   7%|▋         | 1/15 [01:27<20:30, 87.87s/it]


Epoch 001 train_loss: 2.9789   val_loss 2.5483 train_acc 0.1522 val_acc 0.2642
loss:  2.1421051848593913


epoch:  13%|█▎        | 2/15 [02:39<16:59, 78.39s/it]

...


Epoch 013 train_loss: 0.0777   val_loss 0.7833 train_acc 0.9774 val_acc 0.8491
loss:  0.06894647818475616


epoch:  93%|█████████▎| 14/15 [17:07<01:11, 71.80s/it]


Epoch 014 train_loss: 0.0689   val_loss 0.8390 train_acc 0.9781 val_acc 0.8443
loss:  0.07889919986848933


epoch: 100%|██████████| 15/15 [18:18<00:00, 73.24s/it]


Epoch 015 train_loss: 0.0789   val_loss 0.8125 train_acc 0.9764 val_acc 0.8565


In [ ]:
history = train(alexnet_, train_dataset, val_dataset, epochs = 10, batch_size = 128, lr = 1e-5)

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

loss:  0.01983769296429438


epoch:  10%|█         | 1/10 [01:13<10:57, 73.10s/it]


Epoch 001 train_loss: 0.0198   val_loss 0.8234 train_acc 0.9953 val_acc 0.8679
loss:  0.010605096767297123


epoch:  20%|██        | 2/10 [02:25<09:42, 72.81s/it]

...


Epoch 008 train_loss: 0.0042   val_loss 0.9839 train_acc 0.9989 val_acc 0.8682
loss:  0.004333139433631505


epoch:  90%|█████████ | 9/10 [10:50<01:12, 72.20s/it]


Epoch 009 train_loss: 0.0043   val_loss 0.9474 train_acc 0.9987 val_acc 0.8717
loss:  0.00427050603567612


epoch: 100%|██████████| 10/10 [12:03<00:00, 72.30s/it]


Epoch 010 train_loss: 0.0043   val_loss 0.9818 train_acc 0.9989 val_acc 0.8689


**AlexNet Predict**

In [ ]:
label_encoder = pickle.load(open("label_encoder.pkl", 'rb'))

In [ ]:
test_dataset = SimpsonsDataset(test_files, mode="test")
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=128)
probs = predict(alexnet_, test_loader)


preds = label_encoder.inverse_transform(np.argmax(probs, axis=1))
test_filenames = [path.name for path in test_dataset.files]

In [ ]:
import pandas as pd
#my_submit = pd.read_csv("gdrive/MyDrive/simpsons/data/labels.csv")
my_submit = pd.DataFrame({'Id': test_filenames, 'Expected': preds})
my_submit.head()

,Id,Expected
0,img0.jpg,nelson_muntz
1,img1.jpg,bart_simpson
2,img10.jpg,ned_flanders
3,img100.jpg,chief_wiggum
4,img101.jpg,apu_nahasapeemapetilon


In [ ]:
my_submit.to_csv('gdrive/MyDrive/alexnet.csv', index=False)

In [ ]:
torch.cuda.empty_cache()